## BLERSSI Pipeline Deployment

Model training Using Cisco UCS server and Model deployment in GCP Al platform

## Authorizing  gcloud with a service account

To authorize gcloud to access Google Cloud Platform using an existing service account while also specifying a project, run:

In [1]:
!gcloud auth activate-service-account <<ACCOUNT>> --key-file=auth.json --project=<<PROJECT-ID>>

Activated service account credentials for: [<<ACCOUNT>>]


## Install libraries

In [2]:
!pip install kfp --user

     |████████████████████████████████| 119 kB 9.7 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 4.6 MB/s  eta 0:00:01
     |████████████████████████████████| 82 kB 3.8 MB/s  eta 0:00:01
  Created wheel for kfp: filename=kfp-0.5.1-py3-none-any.whl size=163151 sha256=f47c2b6e842d2aa86ec280008f3933a6309a7432bb2be94dcd32c044964c66b1
  Stored in directory: /home/jovyan/.cache/pip/wheels/2f/26/f9/e3836cb6e6cabd63ef912304e18a852ac29cb870a4a0b85f98
  Created wheel for kfp-server-api: filename=kfp_server_api-0.5.0-py3-none-any.whl size=106319 sha256=9873f264a8cbbb780d43d2d979126a6ca2d062e6920b3ff14b4ff88966a5acd8
  Stored in directory: /home/jovyan/.cache/pip/wheels/73/36/4e/bfe2efeeea4f74f04984ebe1d44136202b72191302f4760951
  Created wheel for strip-hints: filename=strip_hints-0.1.9-py2.py3-none-any.whl size=24671 sha256=138d66f1812ab49f53e18513549c5e12ffb36e35970243b8401afe381fb8ea62
  Stored in directory: /home/jovyan/.cache/pip/wheels/21/6d/fa/7ed7c0560e1ef39ebabd5cc0241e7

## Restart Notebook Kernel

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

## Import libraries

In [1]:
import time
import os
from datetime import datetime
import logging
import kfp
import kfp.gcp as gcp
import kfp.dsl as dsl
from kfp import components
from kubernetes import client as k8s_client
import googleapiclient.discovery


logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

%env GOOGLE_APPLICATION_CREDENTIALS=auth.json

env: GOOGLE_APPLICATION_CREDENTIALS=auth.json


## Set Pipeline parameters

In [2]:
# execution_mode (string): where the notebook is being run
# Sample: 'local', 'in-cluster'
execution_mode = 'in-cluster'

# host (string): KF Pipelines service endpoint
# Sample:  "http://10.10.10.10:31380/pipeline"
host = ""

# bucket_name (string): GCP bucket to be used by the pipeline
# Sample: "blerssi-model-store"
bucket_name = ""

# gcp_secret_name (string): GCP secret where IAM creds are stored
# Sample: 'user-gcp-sa'
gcp_secret_name = 'user-gcp-sa'

gcp_region = "us-central1"

timestamp = datetime.now().strftime("%d_%m_%y_%H_%M_%S")
logging.info("timestamp for pipeline artifacts: %s", timestamp)

# model name to create a re-usable AL Platform Model resource
model_name = 'blerssi_tf_model_'+timestamp

version_name = "blerssi_model_"+timestamp

# model artifact URL
# Path to the model
model_version = !(date +%s)
model_path = 'gs://'+bucket_name+'/blerssi/%s/'%model_version[0]

# GCP instance type
instance_type = 'mls1-c1-m2'

#GCP Project Id
project_id=""

INFO:root:timestamp for pipeline artifacts: 21_05_20_06_45_55


In [3]:
if not host:
    print("host parameter is empty, set this parameter")
if not bucket_name:
    print("bucket_name parameter is empty, set this parameter")
if not gcp_region:
    print("gcp_region parameter is empty, set this parameter")
if not project_id:
    print("project_id parameter is empty, set this parameter")

## Set deploy component yaml path variables

In [4]:
deploy="https://raw.githubusercontent.com/kubeflow/pipelines/01a23ae8672d3b18e88adf3036071496aca3552d/components/gcp/ml_engine/deploy/component.yaml"
if execution_mode == "local" and host == None:
    raise ValueError("Please set host to the appropriate URL")
elif execution_mode != "local":
    execution_mode = "in-cluster"

## Define pipeline functions

In [5]:
mlengine_deploy_op = components.load_component_from_url(deploy)

def blerssi_train_upload_op(step_name='blerssi-train'):
    return dsl.ContainerOp(
        name='blerssi-train-upload-gcp',
        image='docker.io/samba07/blerssi-gcp-mlengine:0.2',
        command=['python', '/opt/blerssi-model.py'],
        arguments=['--bucket-name', bucket_name,
                   '--model-version', model_version[0]]
    ).apply(gcp.use_gcp_secret(gcp_secret_name))

In [6]:
import kfp.dsl as dsl
import json
@dsl.pipeline(
    name='CloudML deploy pipeline',
    description='CloudML deploy pipeline'
)
def pipeline(
      model_uri = '',
      project_id = '',
      model_id = '',
      version_id = '',
      runtime_version = '1.10',
      python_version = '',
      version = '',
      replace_existing_version = 'False',
      set_default = 'True',
      wait_interval = '30'):
    
    train_upload_model = blerssi_train_upload_op()
    
    task = mlengine_deploy_op(
        model_uri=model_uri, 
        project_id=project_id, 
        model_id=model_id, 
        version_id=version_id, 
        runtime_version=runtime_version, 
        python_version=python_version,
        version=version, 
        replace_existing_version=replace_existing_version, 
        set_default=set_default, 
        wait_interval=wait_interval).apply(gcp.use_gcp_secret(gcp_secret_name))
    task.after(train_upload_model)

In [7]:
try:
    import kfp.compiler as compiler
    compiler.Compiler().compile(pipeline, 'blerssi_pipeline.tar.gz')
except RuntimeError as err:
    logging.debug(err)
    logging.info("Argo workflow failed validation check but it can still be used to run experiments.")

## Create kubeflow pipeline experiment

In [8]:
client = None
if execution_mode == "local":
    client = kfp.Client(host=host)
else:
    client = kfp.Client()
blerssi_hybrid_experiment = client.create_experiment(name='BLERSSI-GCP')

## Create kubeflow pipeline run

In [9]:
run = client.run_pipeline(blerssi_hybrid_experiment.id, 'blerssi-gcp-mlengine-pipeline-'+timestamp, pipeline_package_path='blerssi_pipeline.tar.gz',
                         params={'model_uri': model_path,
                                 'project_id': project_id,
                                 'model_id': model_name,
                                 'version_id': version_name})

## Model validation with Al platform Endpoint

In [10]:
service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}/versions/{}'.format(project_id, model_name, version_name)

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exceptio

## Send prediction API request

In [11]:
instances=[{"b3001":[1.0] , "b3002":[1.0] , "b3003":[0.4], "b3004":[1.0] , "b3005":[0.385] , "b3006":[0.280] , "b3007":[0.405] , "b3008":[1.0] , "b3009":[1.0] , "b3010":[1.0] , "b3011":[1.0] , "b3012":[1.0] , "b3013":[1.0]}]
response = service.projects().predict(
    name=name,
    body={'instances': instances}
).execute()

if 'error' in response:
    raise RuntimeError(response['error'])
else:
  print(response['predictions'])

INFO:googleapiclient.discovery:URL being requested: POST https://ml.googleapis.com/v1/projects/cpsg-ai-kubeflow/models/blerssi_tf_model_21_05_20_06_14_05/versions/blerssi_model_21_05_20_06_14_05:predict?alt=json


[{'all_class_ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104], 'all_classes': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', 

## Cleaning up
To clean up all GCP resources used in this Notebook

In [12]:
# Delete version resource
! gcloud ai-platform versions delete $version_name --quiet --model $model_name 

# Delete model resource
! gcloud ai-platform models delete $model_name --quiet

# Delete Cloud Storage objects that were created
! gsutil -m rm -r gs://$bucket_name/

Deleting version [blerssi_model_21_05_20_06_14_05]......done.                  
Deleting model [blerssi_tf_model_21_05_20_06_14_05]...done.                    
Removing gs://ai-platform-artifact/blerssi/1590038786/saved_model.pb#1590038597864770...
Removing gs://ai-platform-artifact/blerssi/1590038786/variables/variables.data-00000-of-00002#1590038597562244...
Removing gs://ai-platform-artifact/blerssi/1590041645/variables/variables.data-00000-of-00002#1590041448802719...
Removing gs://ai-platform-artifact/blerssi/1590038786/variables/variables.data-00001-of-00002#1590038597087457...
Removing gs://ai-platform-artifact/blerssi/1590038786/variables/variables.index#1590038597360202...
Removing gs://ai-platform-artifact/blerssi/1590041645/variables/variables.data-00001-of-00002#1590041448428046...
Removing gs://ai-platform-artifact/blerssi/1590041645/saved_model.pb#1590041449032932...
Removing gs://ai-platform-artifact/blerssi/1590041645/variables/variables.index#1590041448627018...
/ [8/8